In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets

import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, DataCollatorForSeq2Seq, get_scheduler
from tqdm import tqdm

In [ ]:
from transformers import (
    MT5Tokenizer,
    MT5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    get_scheduler
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_excel('/content/drive/MyDrive/CS549_Project/CS549_project.xlsx')
df = df[['Stock_Report', 'News']]
df["Stock_Report"] = "haberleştir: " + df["Stock_Report"]

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)

def tokenize_function(examples):
    model_inputs = tokenizer(examples['Stock_Report'], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['News'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['Stock_Report', 'News'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['Stock_Report', 'News'])

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)
num_epochs = 40
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=500, num_training_steps=num_epochs * len(train_loader))

save_path = "best_mt5_model.pt"

best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    print(f"\n🟢 Epoch {epoch+1}/{num_epochs}")

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📉 Training Loss     : {avg_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"🧪 Validation Loss  : {avg_val_loss:.4f}")

    # En iyi modeli yine kaydediyoruz (her epoch'ta güncellensin istersen kaldırabilirsin)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)
        print(f"✅ Yeni en iyi model kaydedildi (Loss: {avg_val_loss:.4f})")

print(f"\n🏁 Eğitim tamamlandı. En iyi validation loss: {best_val_loss:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]


🟢 Epoch 1/40


...

🧪 Validation Loss  : 2.2403

🟢 Epoch 40/40


Training: 100%|██████████| 45/45 [00:18<00:00,  2.46it/s]


📉 Training Loss     : 0.5920


Validating: 100%|██████████| 5/5 [00:00<00:00,  8.12it/s]

🧪 Validation Loss  : 2.2422

🏁 Eğitim tamamlandı. En iyi validation loss: 1.8830


In [ ]:
def generate_news(stock_report_text, model, tokenizer, device):
    prompt = "haberleştir: " + stock_report_text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            num_beams=4,
            #no_repeat_ngram_size=3,
            #length_penalty=1.2,
            repetition_penalty=2.5,
            early_stopping=True
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

sample_stock_report = "Şirketimizin 21.03.2025 tarihli özel durum açıklamasında, paylarının tamamına sahip olduğumuz bağlı ortaklığımız Marbaş Menkul Değerler AŞ'nin (Marbaş Menkul), 150.000.000 TL olan çıkarılmış sermayesinin 360.000.000 TL'ye çıkarılmasına, artırılan 210.000.000 TL sermayenin iç kaynaklardan karşılanmasına ilişkin olarak Marbaş Menkul tarafından Sermaye Piyasası Kurulu'na yapılan başvurunun olumlu karşılandığı duyurulmuştur."

generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)

Üretilen Haber:
Şirket, tamamına sahip olduğu bağlı ortaklığı Marbaş Menkul Değerler A.Ş.'nin sermaye artırımına ilişkin önemli bir adım attı. 150 milyon TL olan çıkarılmış sermayeyi iç kaynaklardan karşılamaya yönelik Sermaye Piyasası Kurulu’na (SPK) yapılan başvuru olumlu karşılandı. Bu artışın tamamı, yatırımcılar açısından güçlü bir büyüme adımı olarak değerlendiriliyor. Şirketin önümüzdeki dönemde finansal yapısını güçlendirme stratejisinin bir parçası gibi duruyor.


In [ ]:
def generate_news(stock_report_text, model, tokenizer, device):
    prompt = "haberleştir: " + stock_report_text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            num_beams=4,
            no_repeat_ngram_size=3,
            length_penalty=1.2,
            early_stopping=True
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

# Örnek üretim
sample_stock_report = "Şirketimizin 21.03.2025 tarihli özel durum açıklamasında, paylarının tamamına sahip olduğumuz bağlı ortaklığımız Marbaş Menkul Değerler AŞ'nin (Marbaş Menkul), 150.000.000 TL olan çıkarılmış sermayesinin 360.000.000 TL'ye çıkarılmasına, artırılan 210.000.000 TL sermayenin iç kaynaklardan karşılanmasına ilişkin olarak Marbaş Menkul tarafından Sermaye Piyasası Kurulu'na yapılan başvurunun olumlu karşılandığı duyurulmuştur."

generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)

Üretilen Haber:
Şirket, 21 Mart 2025 tarihli özel durum açıklamasında duyurduğu 150 milyon TL olan çıkarılmış sermayesini, iç kaynaklardan karşılamaya yönelik önemli bir adım daha attı. 15 milyar TL olan sermaye, Marbaş Menkul Döğer A.Ş.’den 360 milyon Euro’ya çıkarılıyor. Bu artışın tamamı dış kaynaklardan karşılanacak. Sermaye Piyasası Kurulu’na yapılan başvuru da olumlu karşılandı. Bu gelişmeyle birlikte Marbaş’ın önümüzdeki yıllarda yaptığı yatırımlarla ilgili stratejik bir hamle oldu.


In [ ]:
def generate_news(stock_report_text, model, tokenizer, device):
    prompt = "haberleştir: " + stock_report_text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            num_beams=4,
            #no_repeat_ngram_size=3,
            #length_penalty=1.2,
            repetition_penalty=2.5,
            early_stopping=True
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
sample_stock_report = "Şirketimiz, iştiraki Colendi Menkul’ün sermayesini 110 milyon TL’ye çıkarma işlemine katılarak %30,51 oranında ortak olmuştur. Ayrıca 1.8 milyon USD ve 12.8 milyon TL ödemeyle yapılacak pay devri sonrası şirketteki pay oranı %51'e yükselecektir."

generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)

Üretilen Haber:
Şirket, iştiraki Colendi Menkul’ün sermaye artırımı sürecine katılarak şirkette %30,51 oranında ortaklık sağladı. Yapılacak olan 1,8 milyon USD ve 12,8 milyar TL tutarındaki yeni pay devri sonrası şirketteki pay oranı %51’e yükselecek. Bu hamle, şirketin büyüme hedefleri doğrultusunda ilerleyen dönemde daha güçlü bir finansal yapıya kavuşduğunu gösteriyor.


In [ ]:
sample_stock_report ="Sermayesine %49 oranında ortaklığımızın bulunduğu iştiraklerimiz Sour Turizm A.Ş. ve Çağrankaya Turizm A.Ş.'nin sermayelerinin 250.000.000 TL'den 400.000.000 TL'ye çıkarılmasına karar verilmiştir."

generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)

Üretilen Haber:
Şirket, büyüme planlarını hızlandırmak için önemli bir adımı daha tamamladı. %49 oranında ortaklığı bulunan Asya merkezli turizm şirketlerinden Sarom ve Çağrankaya Turizm A.Ş.’nin sermayesi 250 milyon TL’den 40  milyar TL'ye çıkarıldı. Bu hamle, grup içi yatırımlar açısından oldukça değerli bir gelişme gibi duruyor.
